In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("/content/drive/MyDrive/players_data_full_realistic.csv")

target_player_id = df.iloc[59]["Player_ID"]
target_player = df[df["Player_ID"] == target_player_id].iloc[0]

filtered_df = df[df["Sport"] == target_player["Sport"]].copy()
filtered_df = filtered_df[filtered_df["Player_ID"] != target_player_id]

numeric_features = ["Age", "Hours_Available"]
categorical_features = [
    "City", "Neighborhood", "Skill_Level", "Competitive_Level",
    "Available_Days", "Available_Times"
]

scaler = MinMaxScaler()
scaled_numeric = scaler.fit_transform(filtered_df[numeric_features])

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(filtered_df[categorical_features])

full_data = np.hstack([scaled_numeric, encoded_cats])

target_numeric = scaler.transform([target_player[numeric_features]])
target_cats = encoder.transform([target_player[categorical_features]])
target_full = np.hstack([target_numeric, target_cats])

similarities = cosine_similarity(target_full.reshape(1, -1), full_data)[0]
filtered_df["Similarity"] = similarities

top_matches = filtered_df.sort_values(by="Similarity", ascending=False).head(5)

print("أفضل المنافسين للاعب:", target_player["Player_Name"])
print(top_matches[[
    "Player_ID", "Player_Name", "Sport", "Skill_Level", "Neighborhood",
    "Available_Days", "Available_Times", "Similarity"
]])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/players_data_full_realistic.csv")

target_player_id = df.iloc[59]["Player_ID"]
target_player = df[df["Player_ID"] == target_player_id].iloc[0]

filtered_df = df[(df["Sport"] == target_player["Sport"]) &
                 (df["Gender"] == target_player["Gender"]) &
                 (df["City"] == target_player["City"])].copy()
filtered_df = filtered_df[filtered_df["Player_ID"] != target_player_id]

numeric_features = ["Age", "Hours_Available"]
categorical_features = [
    "City", "Neighborhood", "Skill_Level", "Competitive_Level",
    "Available_Days", "Available_Times", "Gender"
]

scaler = MinMaxScaler()
scaled_numeric = scaler.fit_transform(filtered_df[numeric_features])

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(filtered_df[categorical_features])

full_data = np.hstack([scaled_numeric, encoded_cats])

target_numeric = scaler.transform([target_player[numeric_features]])
target_cats = encoder.transform([target_player[categorical_features]])
target_full = np.hstack([target_numeric, target_cats])

X = full_data
y = np.ones(full_data.shape[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

predictions = model.predict(X)

filtered_df["Predicted_Similarity"] = predictions

top_matches = filtered_df.sort_values(by="Predicted_Similarity", ascending=False).head(5)

print("أفضل المنافسين للاعب:", target_player["Player_Name"])
print(top_matches[[
    "Player_ID", "Player_Name", "Sport", "Skill_Level", "City", "Neighborhood",
    "Available_Days", "Available_Times", "Gender", "Predicted_Similarity"
]])
